<a href="https://colab.research.google.com/github/PatrickRuan/3rd-ML100Days/blob/master/homework/Day_012_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

root = 'drive/My Drive/'
learning = root + '學習/'

# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = learning + 'Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [3]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [39]:
import warnings
warnings.filterwarnings("ignore")

# 空值補 -1, 做羅吉斯迴歸
t_ls = []
df_m11 = df.fillna(-1)
train_X = df_m11[:train_num]
estimator = LogisticRegression()
titanic=cross_val_score(estimator, train_X, train_Y, cv=5).mean()

t_ls.append(titanic)

t_ls

[0.6960299128976762]

In [40]:
# 空值補 0, 做羅吉斯迴歸


df_ml0 = df.fillna(0)
train_X = df_ml0[:train_num]
model = LogisticRegression()
cross_val_score(model, train_X, train_Y, cv=5).mean()


t_ls.append(titanic)
t_ls

[0.6960299128976762, 0.6960299128976762]

In [41]:
df_mlm = df.fillna(df.mean())
train_X = df_mlm[:train_num]
model = LogisticRegression()
titanic = cross_val_score(model, train_X, train_Y, cv=5).mean()


t_ls.append(titanic)

t_ls

[0.6960299128976762, 0.6960299128976762, 0.6981761033723469]

In [42]:

for n,i in zip(range(len(t_ls)),t_ls):
  print(f'the {i} items of estimation is {t_ls[n]:8.2%}')

the 0.6960299128976762 items of estimation is   69.60%
the 0.6960299128976762 items of estimation is   69.60%
the 0.6981761033723469 items of estimation is   69.82%


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [0]:
dfs=df.fillna(df.mean())
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mms = MinMaxScaler()
stds = StandardScaler()
df_stds = stds.fit_transform(dfs)
df_mms =mms.fit_transform(dfs)

In [0]:
titanic = cross_val_score(model, df_stds[:train_num],train_Y, cv=5).mean()
t_ls.append(titanic)
titanic = cross_val_score(model, df_mms[:train_num],train_Y, cv=5).mean()
t_ls.append(titanic)

In [47]:
for n,s in enumerate(t_ls):
  print(f'the {n+1}_th score is, {s:8.2%}')
  

the 1_th score is,   69.60%
the 2_th score is,   69.60%
the 3_th score is,   69.82%
the 4_th score is,   69.59%
the 5_th score is,   69.94%
